<a href="https://colab.research.google.com/github/Pedro-dev-083/TopicosDeBigDataEmPython-Est-cio/blob/main/Tema_3_Princ%C3%ADpios_de_Desenvolvimento_de_Spark_com_Python_Est%C3%A1cio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação do ambiente - Aula 3.2.1

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Mudei o link de download, porque o da Estácio está quebrado



In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz

Adicionei esse código para descompactar, pois no da estácio não tinha

In [3]:
!tar xf spark-3.2.4-bin-hadoop3.2.tgz

sample_data		   spark-3.2.4-bin-hadoop3.2.tgz
spark-3.2.4-bin-hadoop3.2  spark-3.2.4-bin-hadoop3.2.tgz.1


In [4]:
!pip install -q findspark

In [5]:
!pip install -q pyspark

Retirei o .tgz, porque o objetivo é acessar o diretório, e não o arquivo zipado

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop3.2"

# Começando a trabalhar com o Spark - Aula 3.2.2

In [10]:
import findspark
findspark.init()

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [12]:
dataset = spark.read.csv('/content/sample_data/california_housing_test.csv',inferSchema=True, header =True)

In [ ]:
dataset.printSchema()

# Convertendo Pandas DataFrame para Spark DataFrame - Aula 3.3.3

In [15]:
import pandas as pd
import numpy as np
media = 0
desvio_padrao=0.1
pd_temporario = pd.DataFrame(np.random.normal(media,desvio_padrao,100))
spark_temporario = spark.createDataFrame(pd_temporario)
print(spark.catalog.listTables())
spark_temporario.createOrReplaceTempView("nova_tabela_temporaria")
print(spark.catalog.listTables())
spark.stop()

/content/spark-3.2.4-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


[Table(name='nova_tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
[Table(name='nova_tabela_temporaria', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


# Práticas com MapReduce - Aula 3.4.1

- Exemplo 1

In [30]:
from pyspark import SparkContext
spark_contexto = SparkContext()

In [31]:
vetor = np.array([10,20,30,40,50])

In [32]:
paralelo = spark_contexto.parallelize(vetor)

In [33]:
mapa = paralelo.map(lambda x : x**4-10*x**2+3)

In [34]:
mapa.collect()

[9003, 156003, 801003, 2544003, 6225003]

- Exemplo 2

In [21]:
paralelo = spark_contexto.parallelize(["distribuida", "distribuida", "spark", "rdd", "spark", "spark"])

In [22]:
funcao_lambda = lambda x:(x,1)

In [23]:
from operator import add
mapa = paralelo.map(funcao_lambda).reduceByKey(add).collect()

In [24]:
for (w, c) in mapa:
  print("{}:{}".format(w, c))

distribuida:2
spark:3
rdd:1


In [25]:
spark_contexto.stop()

# Exemplo prático de transformação e ação - Aula 3.4.2

In [36]:
lista = [1, 2, 3, 4, 5, 3]
lista_rdd = spark_contexto.parallelize(lista)

In [37]:
lista_rdd.count()

6

In [38]:
par_ordenado = lambda numero: (numero, numero*10)

In [39]:
lista_rdd.flatMap(par_ordenado).collect()

[1, 10, 2, 20, 3, 30, 4, 40, 5, 50, 3, 30]

In [40]:
lista_rdd.map(par_ordenado).collect()

[(1, 10), (2, 20), (3, 30), (4, 40), (5, 50), (3, 30)]

In [41]:
spark_contexto.stop()